In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c anaconda xlrd --yes

import folium 
import xlrd
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

print('Folium installed and imported!')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0rc0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0rc0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  conda-forge --> 2019.9.11-py36_0 anaconda
    openssl: 1.1.1d-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0 anaconda


cer

In [3]:
df_peru = pd.read_excel(open('geodir-ubigeo-peru.xlsx', 'rb'),sheet_name='ubigeo_inei') 

print('Data downloaded and read into a dataframe!')

df_peru['Distrito'] = df_peru.Distrito.astype(str)
df_peru['Provincia'] = df_peru.Provincia.astype(str)

#Group by Departament
df_peru_data=df_peru.groupby('Departamento',as_index=False).sum()
df_peru_data=df_peru_data.apply(lambda x: x.astype(str).str.upper())

#Convert field to float
df_peru_data['Poblacion'] = df_peru_data.Poblacion.astype(float)
df_peru_data['Superficie'] = df_peru_data.Superficie.astype(float)
df_peru_data['Y'] = df_peru_data.Y.astype(float)
df_peru_data['X'] = df_peru_data.X.astype(float)

#Calculate Poblation per Kilometer
df_peru_data['PoblacionxKm2']=df_peru_data['Poblacion']/df_peru_data['Superficie']

#Exclude Lima and Callao Departament to analyses other Department.
df_peru_data=df_peru_data.drop(df_peru_data.index[14])
df_peru_data=df_peru_data.drop(df_peru_data.index[6])
df_peru_data

Data downloaded and read into a dataframe!


,Departamento,Ubigeo,Poblacion,Superficie,Y,X,PoblacionxKm2
0,AMAZONAS,872418,425832.0,39249.13,-513.09100,-6546.49210,10.849463
1,ANCASH,3500411,1166180.0,35881.30,-1547.78840,-12878.17160,32.501052
2,APURIMAC,2553828,464583.0,21037.99,-1169.15150,-6139.68930,22.083051
3,AREQUIPA,4402800,1329801.0,63343.95,-1740.19090,-7886.44640,20.993339
4,AYACUCHO,6022013,711058.0,44357.00,-1653.16360,-8804.64560,16.030345
5,CAJAMARCA,7705419,1540009.0,33304.32,-840.59490,-9994.51770,46.240518
7,CUSCO,9041166,1338897.0,82694.66,-1535.97104,-8051.65712,16.190852
8,HUANCAVELICA,9042953,505495.0,22159.22,-1286.26270,-7491.87530,22.811949
9,HUANUCO,8446193,878196.0,37867.36,-807.24810,-6417.32250,23.191371
10,ICA,4741337,810214.0,21305.51,-600.35930,-3256.05840,38.028379


In [5]:
peru_geo = r'peru_departamental_simple.geojson'

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df_peru_data['Poblacion'].min(),
                              df_peru_data['Poblacion'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# let Folium determine the scale.
#peru_map = folium.Map(location=[-12,-77], zoom_start=5, tiles='Mapbox Bright')
peru_map = folium.Map(location=[-12,-77], zoom_start=5)

peru_map.choropleth(
    geo_data=peru_geo,
    data=df_peru_data,
    columns=['Departamento', 'Poblacion'],
    key_on='feature.properties.NOMBDEP',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population',
    reset=True
)

peru_map

In [6]:
df_atenciones = pd.read_csv("DAT SALUD MENTAL - ATENDIDOS.csv") 

print('Data downloaded and read into a dataframe!')
df_atenciones.head()


df_atenciones['Distrito'] = df_peru.Distrito.astype(str)
df_atenciones['Provincia'] = df_peru.Provincia.astype(str)
df_atenciones['Departamento'] = df_peru.Departamento.astype(str)

#Group by Departament
df_peru_data01=df_atenciones.groupby('Departamento',as_index=False).sum()
df_peru_data01=df_peru_data01.apply(lambda x: x.astype(str).str.upper())

#Convert field to float
df_peru_data01['Atenciones'] = df_peru_data01.Atenciones.astype(float)
df_peru_data01['Atendidos'] = df_peru_data01.Atendidos.astype(float)

#Exclude Lima and Callao Departament to analyses other Department.
df_peru_data01=df_peru_data01.drop(df_peru_data01.index[14])
df_peru_data01=df_peru_data01.drop(df_peru_data01.index[6])


df_peru_data = df_peru_data.rename(columns={'Departamento':'Departamento1'})
df_peru001= pd.concat([df_peru_data01, df_peru_data], axis=1)
df_peru001=df_peru001.drop(['Anio', 'NroMes','ubigeo'], axis=1)
df_peru001=df_peru001.drop(['Ubigeo','Departamento1'], axis=1)
#df_peru001=df_peru001.drop(df_peru001.columns[3], axis=1)

#Calculate Poblation per Kilometer
df_peru001['AtencionXPoblacion']=df_peru001['Atenciones']/df_peru001['Poblacion']*10000
df_peru001

Data downloaded and read into a dataframe!


,Departamento,Atendidos,Atenciones,Poblacion,Superficie,Y,X,PoblacionxKm2,AtencionXPoblacion
0,AMAZONAS,114.0,148.0,425832.0,39249.13,-513.09100,-6546.49210,10.849463,3.475549
1,ANCASH,219.0,283.0,1166180.0,35881.30,-1547.78840,-12878.17160,32.501052,2.426727
2,APURIMAC,322.0,391.0,464583.0,21037.99,-1169.15150,-6139.68930,22.083051,8.416150
3,AREQUIPA,279.0,378.0,1329801.0,63343.95,-1740.19090,-7886.44640,20.993339,2.842531
4,AYACUCHO,206.0,306.0,711058.0,44357.00,-1653.16360,-8804.64560,16.030345,4.303446
5,CAJAMARCA,163.0,220.0,1540009.0,33304.32,-840.59490,-9994.51770,46.240518,1.428563
7,CUSCO,163.0,203.0,1338897.0,82694.66,-1535.97104,-8051.65712,16.190852,1.516173
8,HUANCAVELICA,143.0,192.0,505495.0,22159.22,-1286.26270,-7491.87530,22.811949,3.798257
9,HUANUCO,498.0,584.0,878196.0,37867.36,-807.24810,-6417.32250,23.191371,6.649996
10,ICA,57.0,75.0,810214.0,21305.51,-600.35930,-3256.05840,38.028379,0.925681


In [8]:
peru_geo = r'peru_departamental_simple.geojson'

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df_peru001['AtencionXPoblacion'].min(),
                              df_peru001['AtencionXPoblacion'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# let Folium determine the scale.
peru_map = folium.Map(location=[-12,-77], zoom_start=5)

peru_map.choropleth(
    geo_data=peru_geo,
    data=df_peru001,
    columns=['Departamento', 'AtencionXPoblacion'],
    key_on='feature.properties.NOMBDEP',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population',
    reset=True
)

peru_map

In [9]:
peru_geo = r'peru_departamental_simple.geojson'

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df_peru001['Atenciones'].min(),
                              df_peru001['Atenciones'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# let Folium determine the scale.
peru_map = folium.Map(location=[-12,-77], zoom_start=5)

peru_map.choropleth(
    geo_data=peru_geo,
    data=df_peru001,
    columns=['Departamento', 'Atenciones'],
    key_on='feature.properties.NOMBDEP',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population',
    reset=True
)

peru_map